# Sentiment Classification


## Loading the dataset

In [230]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from keras.datasets import imdb
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras import models, regularizers, layers, optimizers, losses, metrics

vocab_size = 10000 #vocab size

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

In [0]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

## Train test split

In [0]:
#load dataset as a list of ints
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

Inference: From IMDB dataset, vocab size defined 10000 and loaded the dataset.

In [234]:
print(x_train[1])
print(y_train[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    1
  194 1153  194 8255   78  228    5    6 1463 4369 5012  134   26    4
  715    8  118 1634   14  394   20   13  119  954  189  102    5  207
  110 3103   21   14   69  188    8   30   23    7    4  249  126   93
    4  114    9 2300 1523    5  647    4  116    9   35 8163    4  229
    9  340 1322    4  118    9    4  130 4901   19    4 1002    5   89
   29  952   46   37    4  455    9   45   43   38 1543 1905  398    4
 1649 

Inference: Provided data is already pre-processed, means convered into numbers. Instance of x_train[1] checked with its sentiment lable. 

In [235]:
a = (y_train == 0).sum()
print(a)
b = (y_train == 1).sum()
print(b)

12500
12500


In [236]:
a = (y_test == 0).sum()
print(a)
b = (y_test == 1).sum()
print(b)

12500
12500


Inference: As all the sentence has non-uniform lenghts, to make it uniform to maximum lenght 300 words pad_sequence introduced and dataset split in to x_train and x_test. 

In [237]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(25000, 300)
(25000, 300)
(25000,)
(25000,)


## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [238]:
# ref: https://www.kaggle.com/drscarlat/imdb-sentiment-analysis-keras-and-tensorflow

# Get the word index and then Create a key-value pair for word and word_id (12.5 points)

words = imdb.get_word_index()
type(words)



dict

In [239]:

# printing a sample key and value from the words dictionary
print(list(words.keys())[35])
print(list(words.values())[35])


wednesday
16818


In [240]:

# printing 1st 3 set with sorted key
for k in list(sorted(words.keys()))[0:3]:
  print (k,':',words[k])


a : 68893
own : 70879
' : 755


In [241]:

# printing top 15 key:value set from the dictionary
for k in list(words)[0:15]:
  print(k,':',words[k])


fawn : 34701
tsukino : 52006
nunnery : 52007
sonja : 16816
vani : 63951
woods : 1408
spiders : 16115
hanging : 2345
woody : 2289
trawling : 52008
hold's : 52009
comically : 11307
localized : 40830
disobeying : 30568
'royale : 52010


In [242]:

# code to decode the comment
reverse_word_index = dict(
[(value, key) for (key, value) in words.items()])

decoded_review = ' '.join(
[reverse_word_index.get(i - 3, '?') for i in x_train[0]])

print(decoded_review)
print(len(decoded_review))



? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the st

In [0]:
def cut_words(records):
    for lines in records:
        for word in lines:
            word = word[0:50]
    return records

In [244]:
print(cut_words(decoded_review))

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the st

In [245]:
x_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    1,   14,   22,   16,   43,  530,
        973, 1622, 1385,   65,  458, 4468,   66, 3941,    4,  173,   36,
        256,    5,   25,  100,   43,  838,  112,   50,  670,    2,    9,
         35,  480,  284,    5,  150,    4,  172,  112,  167,    2,  336,
        385,   39,    4,  172, 4536, 1111,   17,  546,   38,   13,  447,
          4,  192,   50,   16,    6,  147, 2025,   19,   14,   22,    4,
       1920, 4613,  469,    4,   22,   71,   87,   

In [246]:
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print('---review with words---')
print([id2word.get(i, ' ') for i in x_train[5]])
print('---label---')
print(y_train[5])

---review with words---
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 

In [0]:
import numpy as np
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils, to_categorical
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

model = models.Sequential()
model.add(layers.Dense(100, kernel_regularizer=regularizers.l1(0.001), activation='relu', input_shape=(300,)))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(50, kernel_regularizer=regularizers.l1(0.001), activation='relu'))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(30, kernel_regularizer=regularizers.l1(0.001), activation='relu'))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(16, kernel_regularizer=regularizers.l1(0.001), activation='relu'))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(1, activation='sigmoid'))



In [248]:

model.summary()


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 100)               30100     
_________________________________________________________________
dropout_9 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_10 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 30)                1530      
_________________________________________________________________
dropout_11 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 16)              

In [249]:
x_val = x_train[:5000]
x_train_m = x_train[5000:]
y_val = y_train[:5000]
y_train_m = y_train[5000:]

print("x_val ", x_val.shape)
print("partial_x_train ", x_train_m.shape)
print("y_val ", y_val.shape)
print("partial_y_train ", y_train_m.shape)

epochs = 20
batch_size = 32

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train_m, y_train_m, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val), verbose=2)

results = model.evaluate(x_test, y_test)
print("_"*100)
print("Test Loss and Accuracy")
print("results ", results)
# history_dict = history.history
# history_dict.keys()


x_val  (5000, 300)
partial_x_train  (20000, 300)
y_val  (5000,)
partial_y_train  (20000,)
Train on 20000 samples, validate on 5000 samples
Epoch 1/20
 - 5s - loss: 9.4219 - acc: 0.4994 - val_loss: 9.6470 - val_acc: 0.4908
Epoch 2/20
 - 3s - loss: 9.3483 - acc: 0.5028 - val_loss: 9.6660 - val_acc: 0.4908
Epoch 3/20
 - 3s - loss: 9.2947 - acc: 0.5024 - val_loss: 9.6926 - val_acc: 0.4908
Epoch 4/20
 - 3s - loss: 9.1300 - acc: 0.4979 - val_loss: 9.6670 - val_acc: 0.4908
Epoch 5/20
 - 3s - loss: 8.7823 - acc: 0.5072 - val_loss: 9.7155 - val_acc: 0.4908
Epoch 6/20
 - 3s - loss: 8.6528 - acc: 0.4995 - val_loss: 9.7058 - val_acc: 0.4908
Epoch 7/20
 - 3s - loss: 8.2892 - acc: 0.5010 - val_loss: 9.6797 - val_acc: 0.4908
Epoch 8/20
 - 3s - loss: 7.6600 - acc: 0.5026 - val_loss: 9.6461 - val_acc: 0.4908
Epoch 9/20
 - 3s - loss: 7.3093 - acc: 0.4982 - val_loss: 9.6253 - val_acc: 0.4908
Epoch 10/20
 - 3s - loss: 6.9079 - acc: 0.4985 - val_loss: 9.5562 - val_acc: 0.4908
Epoch 11/20
 - 3s - loss: 5.94

In [250]:
# above defined model is not able to reduce the val_loss.
# So, Trying with LSTM now

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vecor_length, input_length=maxlen))
model.add(LSTM(130))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 300, 32)           320000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 130)               84760     
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 131       
Total params: 404,891
Trainable params: 404,891
Non-trainable params: 0
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 220s 9ms/step - loss: 0.5281 - acc: 0.7404 - val_loss: 0.4773 - val_acc: 0.7847
Epoch 2/5
25000/25000 [==============================] - 215s 9ms/step - loss: 0.3244 - acc: 0.8673 - val_loss: 0.3608 - val_acc: 0.8473
Epoch 3/5
25000/25000 [=========================

In [0]:
import pickle
path= '/content/drive/My Drive/NLP/'
filename = path+'seqNLP1.sav'
pickle.dump(model, open(filename, 'wb'))
 

In [0]:

# load the model from disk
# model = pickle.load(open(filename, 'rb'))

In [253]:
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.04%


In [0]:
y_pred = model.predict(x_test)

In [255]:
y_pred

array([[0.06303275],
       [0.9965552 ],
       [0.92713594],
       ...,
       [0.0446955 ],
       [0.02282029],
       [0.96978694]], dtype=float32)

In [256]:


scores = model.evaluate(x_test, y_test, verbose=0)
print(scores)



[0.3794466183996201, 0.8604]


In [257]:
y_pred = np.where(y_pred > 0.5, 1, 0)
print(y_pred)

[[0]
 [1]
 [1]
 ...
 [0]
 [0]
 [1]]


## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [258]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))



              precision    recall  f1-score   support

           0       0.88      0.83      0.86     12500
           1       0.84      0.89      0.86     12500

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weighted avg       0.86      0.86      0.86     25000



In [259]:
from keras.models import Model

for i, layer in enumerate(model.layers):
  print(i, layer.name)



0 embedding_8
1 lstm_8
2 dense_23


In [260]:
layer_outputs = [layer.output for layer in model.layers]

activation_model = Model(inputs=model.input, outputs=layer_outputs)  # Creates a model that will return these outputs, given the model input

activations = activation_model.predict(x_test) 

first_layer_activation = activations[0]
print(first_layer_activation.shape)

second_layer_activation = activations[1]
print(second_layer_activation.shape)

third_layer_activation = activations[2]
print(third_layer_activation.shape)



(25000, 300, 32)
(25000, 130)
(25000, 1)


In [261]:
activations[2]

array([[0.06303275],
       [0.9965552 ],
       [0.92713594],
       ...,
       [0.0446955 ],
       [0.02282029],
       [0.96978694]], dtype=float32)

In [0]:
words = imdb.get_word_index()

In [263]:
# code to decode the comment from test set
reverse_word_index = dict(
[(value, key) for (key, value) in words.items()])

decoded_review = ' '.join(
[reverse_word_index.get(i - 3, '?') for i in x_test[0]])

print(decoded_review)


? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? please give this one a miss br br ? ? and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite ? so all you madison fans give this a miss


In [264]:
# Prediction from the final layer for the 0th record of test set
activations[2][0]

# based on np.where(y_pred > 0.5, 1, 0) conversion, the value is < 0.5, hence the prediction is -ve for the comment


array([0.06303275], dtype=float32)

In [0]:
pred = np.where(activations[2] > 0.5, 1, 0)

In [266]:
# displaying Prediction for 1st 5 record from test set
for i in range(5):
  print(pred[i])


[0]
[1]
[1]
[0]
[1]


## Conclusion:

LSTM gives higher accuracy for the above project around 86% and can be used for production. LSTM number of hidden layers is 130 which gives the highest accuracy.